# Name: Nhat Minh Dang
# ID: 222172836
# SIT 378 

# Course Detection

## 1) Imported library:

In [3]:
import os
import librosa
import numpy as np
import time 

# 2) Configuration:

In [5]:
AUDIO_PATH = "Sound/Aslan" 
MIN_FREQ = 300  
MAX_FREQ = 2000  
MIN_DURATION = 1  
ANIMAL_THRESHOLD = 0.2 
NUM_RUNS = 100
TOP_DB = 20

# 3) Count the number of files:
This function counts the number of files to verify valid imput.

In [7]:
def count_files_in_directory(directory_path):
    try:
        all_items = os.listdir(directory_path)
        files = [item for item in all_items if os.path.isfile(os.path.join(directory_path, item))]
        return len(files)
    except FileNotFoundError:
        print(f"Directory not found: {directory_path}")
        return 0
    except PermissionError:
        print(f"Permission denied to access: {directory_path}")
        return 0

file_count = count_files_in_directory(AUDIO_PATH)
print(f"Number of files in '{AUDIO_PATH}': {file_count}")


Number of files in 'Sound/Aslan': 45


# 4) Main functions:
This is the main processing steps for Animal Detection. These includes:
- Import audio files with original sample rate and perform Trimming Silence.
- Extract features and use FFT to determine whether the sound is animal or not.
- Record the list of files, executing multiple times and display the accuracy and average execution time.

In [9]:
def list_audio_files(audio_path):
    return [os.path.join(audio_path, f) for f in os.listdir(audio_path) if f.endswith('.wav')]
    
def load_audio(file_path):
    try:
        y, sr = librosa.load(file_path, sr=None)  
        y_trimmed, _ = librosa.effects.trim(y, top_db=TOP_DB)
        return y_trimmed, sr
    except Exception as e:
        print(f"Error loading {file_path}: {e}")
        return None, None

def extract_features(y, sr):
    duration = librosa.get_duration(y=y, sr=sr)
    if duration < MIN_DURATION:
        return None  
    fft = np.abs(np.fft.fft(y))  
    freqs = np.fft.fftfreq(len(fft), d=1/sr)  
    return freqs, fft

def is_animal_sound(freqs, fft):
    valid_indices = (freqs > 0) & (freqs >= MIN_FREQ) & (freqs <= MAX_FREQ)  
    energy_in_range = np.sum(fft[valid_indices]**2)  
    total_energy = np.sum(fft**2)  
    if total_energy == 0:  
        return False
    proportion = energy_in_range / total_energy
    return proportion >= ANIMAL_THRESHOLD

def detect_animal_sounds(audio_path):
    audio_files = list_audio_files(audio_path)
    if not audio_files:
        print("No audio files found in the directory.")
        return [], 0.0

    animal_files = []
    for file in audio_files:
        y, sr = load_audio(file)
        if y is None or sr is None:
            continue
        features = extract_features(y, sr)
        if features is None:
            continue
        freqs, fft = features
        if is_animal_sound(freqs, fft):
            animal_files.append(file)

    accuracy = len(animal_files) / len(audio_files)
    return animal_files, accuracy

# 5) Execution codes:
These are the codes that executes all functions above and display the result.

In [11]:
execution_times = []

for run in range(NUM_RUNS):
    start_time = time.time()  
    
    animal_files, accuracy = detect_animal_sounds(AUDIO_PATH)
    
    end_time = time.time()  
    execution_time = end_time - start_time  
    execution_times.append(execution_time)
    
    print(f"Run {run + 1}:")
    print(f"Animal sound files: {animal_files}")
    print(f"Detection Accuracy: {accuracy * 100:.2f}%")
    print(f"Execution Time: {execution_time:.2f} seconds\n")

average_time = np.mean(execution_times)

print("Execution Times:")
print("-----------------")
for i, exec_time in enumerate(execution_times, 1):
    print(f"Run {i}: {exec_time:.2f} seconds")
print("-----------------")
print(f"Average Execution Time: {average_time:.2f} seconds")

Run 1:
Animal sound files: ['Sound/Aslan\\aslan_10.wav', 'Sound/Aslan\\aslan_12.wav', 'Sound/Aslan\\aslan_13.wav', 'Sound/Aslan\\aslan_14.wav', 'Sound/Aslan\\aslan_16.wav', 'Sound/Aslan\\aslan_17.wav', 'Sound/Aslan\\aslan_19.wav', 'Sound/Aslan\\aslan_2.wav', 'Sound/Aslan\\aslan_20.wav', 'Sound/Aslan\\aslan_21.wav', 'Sound/Aslan\\aslan_22.wav', 'Sound/Aslan\\aslan_23.wav', 'Sound/Aslan\\aslan_24.wav', 'Sound/Aslan\\aslan_26.wav', 'Sound/Aslan\\aslan_27.wav', 'Sound/Aslan\\aslan_28.wav', 'Sound/Aslan\\aslan_29.wav', 'Sound/Aslan\\aslan_30.wav', 'Sound/Aslan\\aslan_31.wav', 'Sound/Aslan\\aslan_32.wav', 'Sound/Aslan\\aslan_33.wav', 'Sound/Aslan\\aslan_35.wav', 'Sound/Aslan\\aslan_38.wav', 'Sound/Aslan\\aslan_4.wav', 'Sound/Aslan\\aslan_40.wav', 'Sound/Aslan\\aslan_41.wav', 'Sound/Aslan\\aslan_42.wav', 'Sound/Aslan\\aslan_43.wav', 'Sound/Aslan\\aslan_44.wav', 'Sound/Aslan\\aslan_7.wav', 'Sound/Aslan\\aslan_9.wav']
Detection Accuracy: 68.89%
Execution Time: 1.97 seconds

Run 2:
Animal sound 